In [4]:
import numpy as np
from numba import njit
@njit(fastmath=True)
def make_xyz(x_start, y_start, z_start, Nbox, Lcell, f_idx, n_queue):
    """
    Calculate phase factor array for the coordinate range we need. Output is
    the x, y, z array of the phase factor, each a 1-D array of length n_queue.
    We use loops to fill an array instead of a meshgrid-reshape-slice approach
    to reduce memory consumption.

    Note
    ----
    In the last loop, f_idx1 + n_queue will exceed Nbox**3, which leads to a
    small waste of memory. This will not affect the result, later we will make
    slice to use only the part up to Nbox**3.
    """
    f_idx1 = 0
    queue_idx = 0
    x_queue = np.empty(n_queue, dtype=np.float32)
    y_queue = np.empty(n_queue, dtype=np.float32)
    z_queue = np.empty(n_queue, dtype=np.float32)
    for i,j,k in np.ndindex(Nbox, Nbox, Nbox):
        if f_idx1 < f_idx:
            f_idx1 += 1
            continue
        elif (f_idx1 >= f_idx) and (queue_idx < n_queue):
            x_queue[queue_idx] = x_start + i * Lcell
            y_queue[queue_idx] = y_start + j * Lcell
            z_queue[queue_idx] = z_start + k * Lcell
            queue_idx += 1
        elif queue_idx >= n_queue:
            break
        else:
            raise Exception('Unexpected behaviour in make_xyz()')

        

    return x_queue, y_queue, z_queue

make_xyz(5, 5, 5, 10, 0.1, f_idx=3, n_queue=10)

1
1
1
1
1
1
1
1
1
1


(array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.], dtype=float32),
 array([5. , 5. , 5. , 5. , 5. , 5. , 5. , 5.1, 5.1, 5.1], dtype=float32),
 array([5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 5. , 5.1, 5.2], dtype=float32))